In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import optuna
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import QuantLib as ql
import mibian
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from copy import deepcopy
from keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Flatten, Dense, TimeDistributed, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from ta import add_all_ta_features
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from tensorflow import keras
keras.utils.set_random_seed(812)
from sklearn.model_selection import KFold

C:\Users\unnia\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\unnia\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\unnia\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\unnia\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Read Data

In [2]:
# Read the data
df = pd.read_csv('data.csv')

## TSLA data preprocessing

In [3]:
df_TSLA = df[df['Stock_TSLA'] == 1.0]

# Calculate Simple Moving Average
df_TSLA['SMA'] = df_TSLA['UNDERLYING_LAST'].rolling(window=14).mean()

# Calculate Exponential Moving Average
df_TSLA['EMA'] = df_TSLA['UNDERLYING_LAST'].ewm(span=14, adjust=False).mean()

# Calculate RSI
df_TSLA['RSI'] = RSIIndicator(close=df_TSLA['UNDERLYING_LAST'], window=14).rsi()

# Calculate Bollinger Bands
indicator_bb = BollingerBands(close=df_TSLA['UNDERLYING_LAST'], window=20, window_dev=2)
df_TSLA['bb_bbm'] = indicator_bb.bollinger_mavg()
df_TSLA['bb_bbh'] = indicator_bb.bollinger_hband()
df_TSLA['bb_bbl'] = indicator_bb.bollinger_lband()

# Realized Volatility
df_TSLA['realized_vol'] = df_TSLA['UNDERLYING_LAST'].pct_change().rolling(window=14).std() * np.sqrt(252)


# For lagged features of the target variable, C_LAST:
df_TSLA['C_LAST_lag1'] = df_TSLA['C_LAST'].shift(1)
# For other variables, you could do the same:
df_TSLA['C_IV_lag1'] = df_TSLA['C_IV'].shift(1)


df_TSLA = df_TSLA[['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST', 'STRIKE', 'DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate', 'dividend_rate','C_LAST']]
df_TSLA = df_TSLA.dropna()

# Find indices where C_IV or DTE is zero
indices_to_remove = df_TSLA[(df_TSLA['C_IV'] == 0) | (df_TSLA['DTE'] == 0)].index

# Drop these indices from the DataFrame
df_TSLA = df_TSLA.drop(indices_to_remove)

# Reset the index
df_TSLA = df_TSLA.reset_index(drop=True)

C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\2518326361.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TSLA['SMA'] = df_TSLA['UNDERLYING_LAST'].rolling(window=14).mean()
C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\2518326361.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TSLA['EMA'] = df_TSLA['UNDERLYING_LAST'].ewm(span=14, adjust=False).mean()
C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\2518326361.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [4]:
# Prepare the features and target variable
X_TSLA = df_TSLA[['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST',  'STRIKE','DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate']].values
y_TSLA = df_TSLA['C_LAST'].values

In [6]:
# Define split points
train_split_TSLA = int(len(X_TSLA) * 0.7)  # 70% of the data for training
val_split_TSLA = int(len(X_TSLA) * 0.8)    

# Split the data into training, validation, and testing sets
X_train_TSLA, X_val_test_TSLA = X_TSLA[:train_split_TSLA], X_TSLA[train_split_TSLA:]
y_train_TSLA, y_val_test_TSLA = y_TSLA[:train_split_TSLA], y_TSLA[train_split_TSLA:]

# Split the remaining 30% into validation and test sets
val_size_TSLA = int(len(X_val_test_TSLA) * (10/30))  
X_val_TSLA, X_test_TSLA = X_val_test_TSLA[:val_size_TSLA], X_val_test_TSLA[val_size_TSLA:]
y_val_TSLA, y_test_TSLA = y_val_test_TSLA[:val_size_TSLA], y_val_test_TSLA[val_size_TSLA:]

## AAPL data preprocessing

In [7]:
df_AAPL = df[df['Stock_AAPL'] == 1.0]

# Calculate Simple Moving Average
df_AAPL['SMA'] = df_AAPL['UNDERLYING_LAST'].rolling(window=14).mean()

# Calculate Exponential Moving Average
df_AAPL['EMA'] = df_AAPL['UNDERLYING_LAST'].ewm(span=14, adjust=False).mean()

# Calculate RSI
df_AAPL['RSI'] = RSIIndicator(close=df_AAPL['UNDERLYING_LAST'], window=14).rsi()

# Calculate Bollinger Bands
indicator_bb = BollingerBands(close=df_AAPL['UNDERLYING_LAST'], window=20, window_dev=2)
df_AAPL['bb_bbm'] = indicator_bb.bollinger_mavg()
df_AAPL['bb_bbh'] = indicator_bb.bollinger_hband()
df_AAPL['bb_bbl'] = indicator_bb.bollinger_lband()

# Realized Volatility
df_AAPL['realized_vol'] = df_AAPL['UNDERLYING_LAST'].pct_change().rolling(window=14).std() * np.sqrt(252)


# For lagged features of the target variable, C_LAST:
df_AAPL['C_LAST_lag1'] = df_AAPL['C_LAST'].shift(1)
# For other variables, you could do the same:
df_AAPL['C_IV_lag1'] = df_AAPL['C_IV'].shift(1)


df_AAPL = df_AAPL[['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST', 'STRIKE', 'DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate', 'C_LAST']]
df_AAPL = df_AAPL.dropna()

# Find indices where C_IV or DTE is zero
indices_to_remove = df_AAPL[(df_AAPL['C_IV'] == 0) | (df_AAPL['DTE'] == 0)].index

# Drop these indices from the DataFrame
df_AAPL = df_AAPL.drop(indices_to_remove)

# Reset the index
df_AAPL = df_AAPL.reset_index(drop=True)

C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\2192086726.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_AAPL['SMA'] = df_AAPL['UNDERLYING_LAST'].rolling(window=14).mean()
C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\2192086726.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_AAPL['EMA'] = df_AAPL['UNDERLYING_LAST'].ewm(span=14, adjust=False).mean()
C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\2192086726.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [8]:
# Prepare the features and target variable
X_AAPL = df_AAPL[['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST',  'STRIKE','DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate']].values
y_AAPL = df_AAPL['C_LAST'].values

In [10]:
# Define split points
train_split_AAPL = int(len(X_AAPL) * 0.7)  # 70% of the data for training
val_split_AAPL = int(len(X_AAPL) * 0.8)    

# Split the data into training, validation, and testing sets
X_train_AAPL, X_val_test_AAPL = X_AAPL[:train_split_AAPL], X_AAPL[train_split_AAPL:]
y_train_AAPL, y_val_test_AAPL = y_AAPL[:train_split_AAPL], y_AAPL[train_split_AAPL:]

# Split the remaining 30% into validation and test sets
val_size_AAPL = int(len(X_val_test_AAPL) * (10/30))  
X_val_AAPL, X_test_AAPL = X_val_test_AAPL[:val_size_AAPL], X_val_test_AAPL[val_size_AAPL:]
y_val_AAPL, y_test_AAPL = y_val_test_AAPL[:val_size_AAPL], y_val_test_AAPL[val_size_AAPL:]

## NVDA data preprocessing

In [11]:
df_NVDA = df[df['Stock_NVDA'] == 1.0]

# Calculate Simple Moving Average
df_NVDA['SMA'] = df_NVDA['UNDERLYING_LAST'].rolling(window=14).mean()

# Calculate Exponential Moving Average
df_NVDA['EMA'] = df_NVDA['UNDERLYING_LAST'].ewm(span=14, adjust=False).mean()

# Calculate RSI
df_NVDA['RSI'] = RSIIndicator(close=df_NVDA['UNDERLYING_LAST'], window=14).rsi()

# Calculate Bollinger Bands
indicator_bb = BollingerBands(close=df_NVDA['UNDERLYING_LAST'], window=20, window_dev=2)
df_NVDA['bb_bbm'] = indicator_bb.bollinger_mavg()
df_NVDA['bb_bbh'] = indicator_bb.bollinger_hband()
df_NVDA['bb_bbl'] = indicator_bb.bollinger_lband()

# Realized Volatility
df_NVDA['realized_vol'] = df_NVDA['UNDERLYING_LAST'].pct_change().rolling(window=14).std() * np.sqrt(252)


# For lagged features of the target variable, C_LAST:
df_NVDA['C_LAST_lag1'] = df_NVDA['C_LAST'].shift(1)
# For other variables, you could do the same:
df_NVDA['C_IV_lag1'] = df_NVDA['C_IV'].shift(1)


df_NVDA = df_NVDA[['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST', 'STRIKE', 'DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate', 'C_LAST']]
df_NVDA = df_NVDA.dropna()

# Find indices where C_IV or DTE is zero
indices_to_remove = df_NVDA[(df_NVDA['C_IV'] == 0) | (df_NVDA['DTE'] == 0)].index

# Drop these indices from the DataFrame
df_NVDA = df_NVDA.drop(indices_to_remove)

# Reset the index
df_NVDA = df_NVDA.reset_index(drop=True)

C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\1211373322.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_NVDA['SMA'] = df_NVDA['UNDERLYING_LAST'].rolling(window=14).mean()
C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\1211373322.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_NVDA['EMA'] = df_NVDA['UNDERLYING_LAST'].ewm(span=14, adjust=False).mean()
C:\Users\unnia\AppData\Local\Temp\ipykernel_91244\1211373322.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [12]:
# Prepare the features and target variable
X_NVDA = df_NVDA[['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST',  'STRIKE','DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate']].values
y_NVDA = df_NVDA['C_LAST'].values

In [13]:
# Define split points
train_split_NVDA = int(len(X_NVDA) * 0.7)  # 70% of the data for training
val_split_NVDA = int(len(X_NVDA) * 0.8)    

# Split the data into training, validation, and testing sets
X_train_NVDA, X_val_test_NVDA = X_NVDA[:train_split_NVDA], X_NVDA[train_split_NVDA:]
y_train_NVDA, y_val_test_NVDA = y_NVDA[:train_split_NVDA], y_NVDA[train_split_NVDA:]

# Split the remaining 30% into validation and test sets
val_size_NVDA = int(len(X_val_test_NVDA) * (10/30))  
X_val_NVDA, X_test_NVDA = X_val_test_NVDA[:val_size_NVDA], X_val_test_NVDA[val_size_NVDA:]
y_val_NVDA, y_test_NVDA = y_val_test_NVDA[:val_size_NVDA], y_val_test_NVDA[val_size_NVDA:]

### Features

In [14]:
# Select features to use for prediction
features = ['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol', 'UNDERLYING_LAST', 'STRIKE', 'DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate']
target = 'C_LAST'

# Black-Scholes-Merton Model

In [16]:
def compute_american_option(row, option_type='call'):
    # Extract parameters from the row
    underlying_price = row['UNDERLYING_LAST']
    strike_price = row['STRIKE']
    interest_rate = row['interest rate'] / 100  # Convert to decimal
    days_to_expiration = row['DTE']
    volatility = row['C_IV'] / 100  # Convert to decimal
    dividend_rate = row['dividend_rate']  # Assuming no dividends, adjust if you have dividend rates

    # Setup the calculation date
    calculation_date = ql.Date.todaysDate()
    ql.Settings.instance().evaluationDate = calculation_date

    # Option data
    maturity_date = calculation_date + int(days_to_expiration)
    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

    # Option type
    if option_type.lower() == 'call':
        option_type = ql.Option.Call
    else:
        option_type = ql.Option.Put

    # Construct the European Option
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.AmericanExercise(calculation_date, maturity_date)
    american_option = ql.VanillaOption(payoff, exercise)

    # Construct the Black-Scholes-Merton process
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(underlying_price))
    flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, interest_rate, day_count))
    dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
    flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, volatility, day_count))
    bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

    # Setup the engine and calculate
    engine = ql.BaroneAdesiWhaleyApproximationEngine(bsm_process)
    american_option.setPricingEngine(engine)

    return american_option.NPV()

## Black-Scholes-Merton for TSLA

In [17]:
# Apply the function to the DataFrame
X_test_df_TSLA = pd.DataFrame(X_test_TSLA, columns=['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST',  'STRIKE','DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate'])
X_test_df_TSLA['BlackScholesPrice'] = X_test_df_TSLA.apply(compute_american_option, axis=1)

# Get the predicted Black-Scholes prices
predicted_prices_TSLA = X_test_df_TSLA['BlackScholesPrice'].values

# Compute the RMSE for Black Sholes Method
rmse_TSLA = mean_squared_error(y_test_TSLA, predicted_prices_TSLA, squared=False)
print(f"RMSE for TSLA dataset: {rmse_TSLA}")

RMSE for TSLA dataset: 21.12744626008425


## Black-Scholes-Merton for AAPL

In [18]:
# Apply the function to the DataFrame
X_test_df_AAPL = pd.DataFrame(X_test_AAPL, columns=['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST',  'STRIKE','DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate'])
X_test_df_AAPL['BlackScholesPrice'] = X_test_df_AAPL.apply(compute_american_option, axis=1)

# Get the predicted Black-Scholes prices
predicted_prices_AAPL = X_test_df_AAPL['BlackScholesPrice'].values

# Compute the RMSE for Black Sholes Method
rmse_AAPL = mean_squared_error(y_test_AAPL, predicted_prices_AAPL, squared=False)
print(f"RMSE for AAPL dataset: {rmse_AAPL}")

RMSE for AAPL dataset: 6.941820707584086


## Black-Scholes-Merton for NVDA

In [19]:
# Apply the function to the DataFrame
X_test_df_NVDA = pd.DataFrame(X_test_NVDA, columns=['C_IV_lag1','C_LAST_lag1','bb_bbm','bb_bbh','bb_bbl','SMA','EMA','RSI', 'realized_vol','UNDERLYING_LAST',  'STRIKE','DTE', 'C_IV', 'C_DELTA', 'C_GAMMA', 'C_VEGA', 'C_THETA', 'C_RHO', 'C_VOLUME', 'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT', 'interest rate','dividend_rate'])
X_test_df_NVDA['BlackScholesPrice'] = X_test_df_NVDA.apply(compute_american_option, axis=1)

# Get the predicted Black-Scholes prices
predicted_prices_NVDA = X_test_df_NVDA['BlackScholesPrice'].values

# Compute the RMSE for Black Sholes Method
rmse_NVDA = mean_squared_error(y_test_NVDA, predicted_prices_NVDA, squared=False)
print(f"RMSE for NVDA dataset: {rmse_NVDA}")

RMSE for NVDA dataset: 44.93740398284506


# Feedforward Neural Network

## Scaling and Converting to Tensor

### TSLA

In [5]:
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train_TSLA)

# Scale the training data
X_train_TSLA_scaled = scaler.transform(X_train_TSLA)

# Scale the validation data
X_val_TSLA_scaled = scaler.transform(X_val_TSLA)

# Scale the test data
X_test_TSLA_scaled = scaler.transform(X_test_TSLA)

# Convert the scaled data to tensors
X_train_TSLA_tensor = torch.tensor(X_train_TSLA_scaled, dtype=torch.float32)
y_train_TSLA_tensor = torch.tensor(y_train_TSLA, dtype=torch.float32).unsqueeze(1)

X_val_TSLA_tensor = torch.tensor(X_val_TSLA_scaled, dtype=torch.float32)
y_val_TSLA_tensor = torch.tensor(y_val_TSLA, dtype=torch.float32).unsqueeze(1)

X_test_TSLA_tensor = torch.tensor(X_test_TSLA_scaled, dtype=torch.float32)
y_test_TSLA_tensor = torch.tensor(y_test_TSLA, dtype=torch.float32).unsqueeze(1)

### AAPL

In [ ]:
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train_AAPL)

# Scale the training data
X_train_AAPL_scaled = scaler.transform(X_train_AAPL)

# Scale the validation data
X_val_AAPL_scaled = scaler.transform(X_val_AAPL)

# Scale the test data
X_test_AAPL_scaled = scaler.transform(X_test_AAPL)

# Convert the scaled data to tensors
X_train_AAPL_tensor = torch.tensor(X_train_AAPL_scaled, dtype=torch.float32)
y_train_AAPL_tensor = torch.tensor(y_train_AAPL, dtype=torch.float32).unsqueeze(1)

X_val_AAPL_tensor = torch.tensor(X_val_AAPL_scaled, dtype=torch.float32)
y_val_AAPL_tensor = torch.tensor(y_val_AAPL, dtype=torch.float32).unsqueeze(1)

X_test_AAPL_tensor = torch.tensor(X_test_AAPL_scaled, dtype=torch.float32)
y_test_AAPL_tensor = torch.tensor(y_tes_AAPL, dtype=torch.float32).unsqueeze(1)

### NVDA

In [ ]:
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train_NVDA)

# Scale the training data
X_train_NVDA_scaled = scaler.transform(X_train_NVDA)

# Scale the validation data
X_val_NVDA_scaled = scaler.transform(X_val_NVDA)

# Scale the test data
X_test_NVDA_scaled = scaler.transform(X_test_NVDA)

# Convert the scaled data to tensors
X_train_NVDA_tensor = torch.tensor(X_train_NVDA_scaled, dtype=torch.float32)
y_train_NVDA_tensor = torch.tensor(y_train_NVDA, dtype=torch.float32).unsqueeze(1)

X_val_NVDA_tensor = torch.tensor(X_val_NVDA_scaled, dtype=torch.float32)
y_val_NVDA_tensor = torch.tensor(y_val_NVDA, dtype=torch.float32).unsqueeze(1)

X_test_NVDA_tensor = torch.tensor(X_test_NVDA_scaled, dtype=torch.float32)
y_test_NVDA_tensor = torch.tensor(y_tes_NVDA, dtype=torch.float32).unsqueeze(1)

### Train Function

In [6]:
# Train the model with early stopping
def train(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, loss_fn, optimizer, epochs=100, batch_size=64, patience=10):
    best_loss = float('inf')
    best_model = None
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        for i in range(0, len(X_train_tensor), batch_size):
            # Get the minibatch
            X_batch = X_train_tensor[i:i+batch_size]
            y_batch = y_train_tensor[i:i+batch_size]

            # Forward pass
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = loss_fn(predictions, y_batch)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        with torch.no_grad():
            val_predictions = model(X_val_tensor)
            val_loss = loss_fn(val_predictions, y_val_tensor).item()

        # Print training and validation loss
        print(f'Epoch {epoch+1}/{epochs} - Loss: {loss.item()}, Val Loss: {val_loss}')

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = deepcopy(model.state_dict())  # Save a copy of the model
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience:
            print('Early stopping triggered')
            break

    # Load the best model state
    model.load_state_dict(best_model)
    return model

### Test Function

In [7]:
# Test the model
def test(model, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
        y_pred = y_pred_tensor.numpy().squeeze()
        y_test_original = y_test_tensor.numpy().squeeze() 
        rmse = mean_squared_error(y_test_original, y_pred, squared=False)
        print(f'Root Mean Squared Error (RMSE): {rmse}')
        return rmse

## Neural Network Model 1

In [9]:
# Neural Network Model
class NN_Model_1(nn.Module):
    def __init__(self, no_features):
        super(NN_Model_1, self).__init__()
        self.mlp_stack = nn.Sequential(
            nn.Linear(no_features, 9),   # One hidden layer with 9 nodes
#             nn.LeakyReLU(0.1),
            nn.ReLU(),
            nn.Linear(9, 1)
        )

    def forward(self, x):
        logits = self.mlp_stack(x)
        return logits

no_features = X_train.shape[1]
model_1 = NN_Model_1(no_features=no_features)

# Weight decay for L2 regularization
weight_decay = 1e-6
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=0.005, weight_decay=weight_decay)
loss_fn_1 = nn.MSELoss()

In [10]:
model_1 = train(model_1, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, loss_fn_1, optimizer_1, 100, 64)

Epoch 1/100 - Loss: 470.0867004394531, Val Loss: 301.2904968261719
Epoch 2/100 - Loss: 538.534912109375, Val Loss: 254.16461181640625
Epoch 3/100 - Loss: 548.2007446289062, Val Loss: 247.9997100830078
Epoch 4/100 - Loss: 558.7402954101562, Val Loss: 226.34312438964844
Epoch 5/100 - Loss: 600.1428833007812, Val Loss: 208.22915649414062
Epoch 6/100 - Loss: 621.6306762695312, Val Loss: 200.08554077148438
Epoch 7/100 - Loss: 633.7864379882812, Val Loss: 195.564453125
Epoch 8/100 - Loss: 650.6800537109375, Val Loss: 192.26658630371094
Epoch 9/100 - Loss: 663.565185546875, Val Loss: 191.4334716796875
Epoch 10/100 - Loss: 669.2100830078125, Val Loss: 191.6699981689453
Epoch 11/100 - Loss: 667.305419921875, Val Loss: 191.7811737060547
Epoch 12/100 - Loss: 662.27880859375, Val Loss: 191.85740661621094
Epoch 13/100 - Loss: 662.814697265625, Val Loss: 191.0597381591797
Epoch 14/100 - Loss: 656.5405883789062, Val Loss: 191.22918701171875
Epoch 15/100 - Loss: 655.5157470703125, Val Loss: 191.264205

In [11]:
test(model_1, X_test_tensor, y_test_tensor)

Root Mean Squared Error (RMSE): 14.060128211975098


14.060128

## Neural Network Model 2

In [9]:
# Neural Network Model
class NN_Model_2(nn.Module):
    def __init__(self, no_features):
        super(NN_Model_2, self).__init__()
        self.mlp_stack = nn.Sequential(
            nn.Linear(no_features, 128),
#             nn.ReLU(),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 128),
            nn.ReLU(),
#             nn.LeakyReLU(0.1),
            nn.Linear(128, 128),
            nn.ReLU(),
#             nn.LeakyReLU(0.1),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        logits = self.mlp_stack(x)
        return logits

no_features = X_train.shape[1]
model = NN_Model_2(no_features=no_features)

# Weight decay for L2 regularization
weight_decay = 1e-6
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=weight_decay)
loss_fn = nn.MSELoss()

In [10]:
model = train(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, loss_fn, optimizer, 100, 32)

Epoch 1/100 - Loss: 2335.32568359375, Val Loss: 489.6029052734375
Epoch 2/100 - Loss: 2190.5810546875, Val Loss: 472.17156982421875
Epoch 3/100 - Loss: 2172.70166015625, Val Loss: 440.4659118652344
Epoch 4/100 - Loss: 2806.53564453125, Val Loss: 404.56103515625
Epoch 5/100 - Loss: 2596.15380859375, Val Loss: 354.4566955566406
Epoch 6/100 - Loss: 2883.505615234375, Val Loss: 262.1570739746094
Epoch 7/100 - Loss: 2876.242431640625, Val Loss: 264.1345520019531
Epoch 8/100 - Loss: 2963.21630859375, Val Loss: 225.7086639404297
Epoch 9/100 - Loss: 3082.59765625, Val Loss: 242.30526733398438
Epoch 10/100 - Loss: 3382.5771484375, Val Loss: 194.72967529296875
Epoch 11/100 - Loss: 3608.9140625, Val Loss: 176.02548217773438
Epoch 12/100 - Loss: 4087.89697265625, Val Loss: 184.01271057128906
Epoch 13/100 - Loss: 3368.0009765625, Val Loss: 170.77850341796875
Epoch 14/100 - Loss: 2748.06396484375, Val Loss: 171.12835693359375
Epoch 15/100 - Loss: 4980.82763671875, Val Loss: 213.86427307128906
Epoch 

In [11]:
test(model, X_test_tensor, y_test_tensor)

Root Mean Squared Error (RMSE): 13.04063606262207


13.040636

# Find Best FNN Model

In [8]:
class BaseNN(nn.Module):
    def __init__(self, no_features, hidden_dims, activation='relu'):
        super(BaseNN, self).__init__()
        layers = []
        input_dim = no_features
        # Define a dictionary to map activation names to their functions
        activations = {
            'relu': nn.ReLU(),
            'tanh': nn.Tanh(),
            'elu': nn.ELU(),
            'leaky_relu': nn.LeakyReLU()
        }
        # Get the selected activation function from the dictionary
        self.activation = activations[activation]
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(self.activation)  # Use the selected activation function
            input_dim = hidden_dim  # Set the input for the next layer
        layers.append(nn.Linear(input_dim, 1))  # Output layer
        self.mlp_stack = nn.Sequential(*layers)

    def forward(self, x):
        logits = self.mlp_stack(x)
        return logits

In [9]:
# Add more hidden dims
hidden_dims_list = [[10], [128], [128, 128]]
batch_sizes = [32, 64, 128]
activation_functions = ['relu', 'elu', 'leaky_relu']

# Keep track of experiments
experiment_log = []

# Experiment with different hyperparameters
for hidden_dims in hidden_dims_list:
    for batch_size in batch_sizes:
        for activation in activation_functions:
            # Initialize model, loss, and optimizer
            model = BaseNN(X_train_tensor.shape[1], hidden_dims, activation)
            criterion = nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-6)

            # Train the model
            model = train(model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, criterion, optimizer, epochs=100, batch_size=batch_size)

            # Test the model
            test_rmse = test(model, X_test_tensor, y_test_tensor)

            # Log the experiment
            experiment_log.append({
                'hidden_dims': hidden_dims,
                'batch_size': batch_size,
                'learning_rate': 0.005,
                'activation': activation,
                'test_rmse': test_rmse
            })


Epoch 1/100 - Loss: 2639.33837890625, Val Loss: 234.24644470214844
Epoch 2/100 - Loss: 3105.52099609375, Val Loss: 197.17481994628906
Epoch 3/100 - Loss: 3298.509765625, Val Loss: 186.71412658691406
Epoch 4/100 - Loss: 3131.5966796875, Val Loss: 188.4255828857422
Epoch 5/100 - Loss: 3090.76806640625, Val Loss: 191.74659729003906
Epoch 6/100 - Loss: 3249.55322265625, Val Loss: 185.41864013671875
Epoch 7/100 - Loss: 3293.562255859375, Val Loss: 182.64285278320312
Epoch 8/100 - Loss: 3004.358154296875, Val Loss: 185.79220581054688
Epoch 9/100 - Loss: 2871.90478515625, Val Loss: 185.39491271972656
Epoch 10/100 - Loss: 2831.16796875, Val Loss: 185.50180053710938
Epoch 11/100 - Loss: 2729.67431640625, Val Loss: 185.38900756835938
Epoch 12/100 - Loss: 2741.638671875, Val Loss: 184.93490600585938
Epoch 13/100 - Loss: 2740.513671875, Val Loss: 184.42054748535156
Epoch 14/100 - Loss: 2742.18017578125, Val Loss: 184.13951110839844
Epoch 15/100 - Loss: 2746.58984375, Val Loss: 183.7508544921875
Ep

Epoch 18/100 - Loss: 2812.33642578125, Val Loss: 220.17176818847656
Epoch 19/100 - Loss: 2805.312255859375, Val Loss: 220.01751708984375
Epoch 20/100 - Loss: 2781.695068359375, Val Loss: 220.1317138671875
Epoch 21/100 - Loss: 2778.307861328125, Val Loss: 219.8286895751953
Epoch 22/100 - Loss: 2773.4814453125, Val Loss: 219.6499481201172
Epoch 23/100 - Loss: 2760.81640625, Val Loss: 219.81126403808594
Epoch 24/100 - Loss: 2761.382568359375, Val Loss: 219.5968475341797
Epoch 25/100 - Loss: 2760.4248046875, Val Loss: 219.49249267578125
Epoch 26/100 - Loss: 2759.624755859375, Val Loss: 219.35621643066406
Epoch 27/100 - Loss: 2760.859375, Val Loss: 219.26699829101562
Epoch 28/100 - Loss: 2757.614013671875, Val Loss: 219.101318359375
Epoch 29/100 - Loss: 2754.271484375, Val Loss: 218.8826141357422
Epoch 30/100 - Loss: 2727.3544921875, Val Loss: 219.17337036132812
Epoch 31/100 - Loss: 2723.29541015625, Val Loss: 219.01220703125
Epoch 32/100 - Loss: 2723.37109375, Val Loss: 218.95458984375
Epo

Epoch 20/100 - Loss: 569.474853515625, Val Loss: 188.01792907714844
Epoch 21/100 - Loss: 573.5648193359375, Val Loss: 187.9072723388672
Early stopping triggered
Root Mean Squared Error (RMSE): 13.600068092346191
Epoch 1/100 - Loss: 419.0306701660156, Val Loss: 323.4391784667969
Epoch 2/100 - Loss: 423.6273193359375, Val Loss: 257.35211181640625
Epoch 3/100 - Loss: 499.48809814453125, Val Loss: 227.14889526367188
Epoch 4/100 - Loss: 595.6652221679688, Val Loss: 210.75523376464844
Epoch 5/100 - Loss: 663.8959350585938, Val Loss: 203.09072875976562
Epoch 6/100 - Loss: 710.160400390625, Val Loss: 198.61212158203125
Epoch 7/100 - Loss: 723.7914428710938, Val Loss: 196.90518188476562
Epoch 8/100 - Loss: 610.822998046875, Val Loss: 203.47203063964844
Epoch 9/100 - Loss: 574.1564331054688, Val Loss: 199.7056121826172
Epoch 10/100 - Loss: 590.3823852539062, Val Loss: 195.51959228515625
Epoch 11/100 - Loss: 580.68701171875, Val Loss: 191.93252563476562
Epoch 12/100 - Loss: 603.5802001953125, Val

Epoch 77/100 - Loss: 685.2701416015625, Val Loss: 194.7994842529297
Epoch 78/100 - Loss: 685.0255126953125, Val Loss: 194.79762268066406
Epoch 79/100 - Loss: 684.7885131835938, Val Loss: 194.7962188720703
Epoch 80/100 - Loss: 684.5596923828125, Val Loss: 194.7947998046875
Epoch 81/100 - Loss: 684.33935546875, Val Loss: 194.7931365966797
Epoch 82/100 - Loss: 684.125244140625, Val Loss: 194.7913360595703
Epoch 83/100 - Loss: 683.9166870117188, Val Loss: 194.78953552246094
Epoch 84/100 - Loss: 683.7131958007812, Val Loss: 194.78770446777344
Epoch 85/100 - Loss: 683.5169677734375, Val Loss: 194.78562927246094
Epoch 86/100 - Loss: 683.326904296875, Val Loss: 194.7836151123047
Epoch 87/100 - Loss: 683.1416015625, Val Loss: 194.7814483642578
Epoch 88/100 - Loss: 682.9609985351562, Val Loss: 194.77906799316406
Epoch 89/100 - Loss: 682.7841186523438, Val Loss: 194.7765350341797
Epoch 90/100 - Loss: 682.6067504882812, Val Loss: 194.77435302734375
Epoch 91/100 - Loss: 682.4351806640625, Val Loss:

Epoch 39/100 - Loss: 2322.251708984375, Val Loss: 163.92938232421875
Epoch 40/100 - Loss: 2304.385498046875, Val Loss: 164.0659637451172
Epoch 41/100 - Loss: 2248.92724609375, Val Loss: 164.1490478515625
Epoch 42/100 - Loss: 2226.27294921875, Val Loss: 164.790283203125
Epoch 43/100 - Loss: 2316.021484375, Val Loss: 163.0209503173828
Epoch 44/100 - Loss: 2297.65625, Val Loss: 162.525146484375
Epoch 45/100 - Loss: 2298.9306640625, Val Loss: 162.6579132080078
Epoch 46/100 - Loss: 2275.71240234375, Val Loss: 162.66038513183594
Epoch 47/100 - Loss: 2279.4306640625, Val Loss: 162.4191436767578
Epoch 48/100 - Loss: 2321.9912109375, Val Loss: 164.94639587402344
Epoch 49/100 - Loss: 2326.8623046875, Val Loss: 162.7255859375
Epoch 50/100 - Loss: 2324.649658203125, Val Loss: 162.62918090820312
Epoch 51/100 - Loss: 2327.140625, Val Loss: 162.42962646484375
Epoch 52/100 - Loss: 2308.43994140625, Val Loss: 162.90687561035156
Epoch 53/100 - Loss: 2311.67724609375, Val Loss: 162.07728576660156
Epoch 5

Epoch 15/100 - Loss: 628.7354125976562, Val Loss: 185.17556762695312
Epoch 16/100 - Loss: 592.046630859375, Val Loss: 182.41860961914062
Epoch 17/100 - Loss: 571.4115600585938, Val Loss: 180.67613220214844
Epoch 18/100 - Loss: 549.7838134765625, Val Loss: 177.31321716308594
Epoch 19/100 - Loss: 561.498779296875, Val Loss: 173.1376495361328
Epoch 20/100 - Loss: 529.28466796875, Val Loss: 169.92340087890625
Epoch 21/100 - Loss: 631.312255859375, Val Loss: 181.67330932617188
Epoch 22/100 - Loss: 621.91064453125, Val Loss: 175.59930419921875
Epoch 23/100 - Loss: 623.437744140625, Val Loss: 174.6270294189453
Epoch 24/100 - Loss: 613.7142944335938, Val Loss: 173.319091796875
Epoch 25/100 - Loss: 597.7915649414062, Val Loss: 170.68862915039062
Epoch 26/100 - Loss: 617.96240234375, Val Loss: 172.65609741210938
Epoch 27/100 - Loss: 630.8218383789062, Val Loss: 172.10537719726562
Epoch 28/100 - Loss: 628.4105834960938, Val Loss: 170.7123565673828
Epoch 29/100 - Loss: 640.041748046875, Val Loss: 

Epoch 67/100 - Loss: 586.9088745117188, Val Loss: 165.7140655517578
Early stopping triggered
Root Mean Squared Error (RMSE): 12.480690002441406
Epoch 1/100 - Loss: 508.23614501953125, Val Loss: 250.04884338378906
Epoch 2/100 - Loss: 609.3339233398438, Val Loss: 217.2535400390625
Epoch 3/100 - Loss: 682.3619995117188, Val Loss: 199.3896484375
Epoch 4/100 - Loss: 739.0493774414062, Val Loss: 189.7615203857422
Epoch 5/100 - Loss: 737.9912719726562, Val Loss: 187.97264099121094
Epoch 6/100 - Loss: 679.3326416015625, Val Loss: 188.36512756347656
Epoch 7/100 - Loss: 668.3851928710938, Val Loss: 181.44515991210938
Epoch 8/100 - Loss: 617.61572265625, Val Loss: 182.5592803955078
Epoch 9/100 - Loss: 621.45263671875, Val Loss: 180.5886688232422
Epoch 10/100 - Loss: 627.4841918945312, Val Loss: 180.2379913330078
Epoch 11/100 - Loss: 617.4655151367188, Val Loss: 185.03021240234375
Epoch 12/100 - Loss: 618.3866577148438, Val Loss: 185.93727111816406
Epoch 13/100 - Loss: 633.7234497070312, Val Loss:

Epoch 14/100 - Loss: 3120.335205078125, Val Loss: 201.8451690673828
Epoch 15/100 - Loss: 3601.901611328125, Val Loss: 177.27923583984375
Epoch 16/100 - Loss: 3187.66748046875, Val Loss: 185.69473266601562
Epoch 17/100 - Loss: 3066.52294921875, Val Loss: 178.44735717773438
Epoch 18/100 - Loss: 2768.85546875, Val Loss: 169.1878204345703
Epoch 19/100 - Loss: 2734.12841796875, Val Loss: 170.0083770751953
Epoch 20/100 - Loss: 2709.28466796875, Val Loss: 184.21229553222656
Epoch 21/100 - Loss: 2636.65576171875, Val Loss: 168.428955078125
Early stopping triggered
Root Mean Squared Error (RMSE): 12.555122375488281
Epoch 1/100 - Loss: 1108.9149169921875, Val Loss: 214.12948608398438
Epoch 2/100 - Loss: 1160.18017578125, Val Loss: 198.72337341308594
Epoch 3/100 - Loss: 840.47998046875, Val Loss: 181.49868774414062
Epoch 4/100 - Loss: 980.7435913085938, Val Loss: 201.0716552734375
Epoch 5/100 - Loss: 847.43359375, Val Loss: 186.85633850097656
Epoch 6/100 - Loss: 947.3255004882812, Val Loss: 197.3

Epoch 15/100 - Loss: 553.49853515625, Val Loss: 176.22073364257812
Epoch 16/100 - Loss: 613.1649169921875, Val Loss: 173.89796447753906
Epoch 17/100 - Loss: 643.5238647460938, Val Loss: 172.80250549316406
Epoch 18/100 - Loss: 608.72412109375, Val Loss: 172.20858764648438
Epoch 19/100 - Loss: 579.13818359375, Val Loss: 170.69613647460938
Epoch 20/100 - Loss: 487.3131408691406, Val Loss: 170.3683319091797
Epoch 21/100 - Loss: 565.1483764648438, Val Loss: 184.26275634765625
Epoch 22/100 - Loss: 526.1085815429688, Val Loss: 187.4327850341797
Early stopping triggered
Root Mean Squared Error (RMSE): 12.293710708618164
Epoch 1/100 - Loss: 1127.519287109375, Val Loss: 234.63795471191406
Epoch 2/100 - Loss: 1052.643310546875, Val Loss: 194.51031494140625
Epoch 3/100 - Loss: 887.8572387695312, Val Loss: 218.49258422851562
Epoch 4/100 - Loss: 939.133056640625, Val Loss: 214.5719757080078
Epoch 5/100 - Loss: 759.5968627929688, Val Loss: 175.40951538085938
Epoch 6/100 - Loss: 771.6891479492188, Val

In [10]:
# Find the best experiment
best_experiment = min(experiment_log, key=lambda x: x['test_rmse'])
print("Best experiment:", best_experiment)

Best experiment: {'hidden_dims': [128], 'batch_size': 32, 'learning_rate': 0.005, 'activation': 'elu', 'test_rmse': 12.186279}


In [ ]:
# Add more hidden dims
hidden_dims_list = [[10], [128], [128, 128]]
batch_sizes = [32, 64, 128]
activation_functions = ['relu', 'leaky_relu']

# Cross-validation setup
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Hyperparameter search loop
for hidden_dims in hidden_dims_list:
    for batch_size in batch_sizes:
        for activation in activation_functions:
            # Cross-validation scores
            cv_scores = []
            
            for train_index, val_index in kf.split(X_train_tensor):
                # Split the data into current training and validation sets for this fold
                X_train_fold, X_val_fold = X_train_tensor[train_index], X_train_tensor[val_index]
                y_train_fold, y_val_fold = y_train_tensor[train_index], y_train_tensor[val_index]
                
                # Initialize model, loss, and optimizer
                model = BaseNN(X_train_fold.shape[1], hidden_dims, activation)
                criterion = nn.MSELoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)
                
                # Train the model on the current fold
                model = train(model, X_train_fold, y_train_fold, X_val_fold, y_val_fold, criterion, optimizer, epochs=100, batch_size=batch_size)
                
                # Evaluate the model on the current validation fold
                val_rmse = test(model, X_val_fold, y_val_fold)
                cv_scores.append(val_rmse)
            
            # Compute the average RMSE over all folds
            average_rmse = np.mean(cv_scores)
            
            # Log the experiment
            experiment_log.append({
                'hidden_dims': hidden_dims,
                'batch_size': batch_size,
                'learning_rate': 0.001,
                'activation': activation,
                'cv_rmse': average_rmse
            })

# Find the best hyperparameter set based on cross-validation RMSE
best_hyperparams = min(experiment_log, key=lambda x: x['cv_rmse'])

In [11]:
experiment_log

[{'hidden_dims': [10],
  'batch_size': 32,
  'learning_rate': 0.005,
  'activation': 'relu',
  'test_rmse': 13.534487},
 {'hidden_dims': [10],
  'batch_size': 32,
  'learning_rate': 0.005,
  'activation': 'elu',
  'test_rmse': 12.903455},
 {'hidden_dims': [10],
  'batch_size': 32,
  'learning_rate': 0.005,
  'activation': 'leaky_relu',
  'test_rmse': 14.67927},
 {'hidden_dims': [10],
  'batch_size': 64,
  'learning_rate': 0.005,
  'activation': 'relu',
  'test_rmse': 14.359043},
 {'hidden_dims': [10],
  'batch_size': 64,
  'learning_rate': 0.005,
  'activation': 'elu',
  'test_rmse': 14.625621},
 {'hidden_dims': [10],
  'batch_size': 64,
  'learning_rate': 0.005,
  'activation': 'leaky_relu',
  'test_rmse': 13.600068},
 {'hidden_dims': [10],
  'batch_size': 128,
  'learning_rate': 0.005,
  'activation': 'relu',
  'test_rmse': 13.122231},
 {'hidden_dims': [10],
  'batch_size': 128,
  'learning_rate': 0.005,
  'activation': 'elu',
  'test_rmse': 14.043781},
 {'hidden_dims': [10],
  'batc